In [3]:
import pandas as pd 
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("../pre-processing/cleaned_dataset.csv")  
df = df[["image_filename", "category", "style", "title", "price", "product_link"]]  

category_mapping = {cat: idx for idx, cat in enumerate(df["category"].unique())}
type_mapping = {typ: idx for idx, typ in enumerate(df["style"].unique())}

df["Category_Label"] = df["category"].map(category_mapping)
df["Type_Label"] = df["style"].map(type_mapping)

np.save("titles.npy", np.array(df["title"].values, dtype=object))
np.save("prices.npy", np.array(df["price"].values, dtype=object))
np.save("links.npy", np.array(df["product_link"].values, dtype=object))

print("✅ Titles, prices, and links saved successfully!")


✅ Titles, prices, and links saved successfully!


In [6]:
image_dir = "../pre-processing/processed_images/"

X, y_category, y_type = [], [], []  

for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row["image_filename"])
    
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  
        img = img / 255.0  

        X.append(img)
        y_category.append(row["Category_Label"])
        y_type.append(row["Type_Label"])
    else:
        print(f"⚠️ Missing: {row['image_filename']}")

X = np.array(X, dtype=np.float32)
y_category = np.array(y_category, dtype=np.int32)
y_type = np.array(y_type, dtype=np.int32)

print(f"\n✅ Loaded {len(X)} images successfully!")


✅ Loaded 240 images successfully!


In [8]:
X_train, X_val, y_train_category, y_val_category, y_train_type, y_val_type = train_test_split(
    X, y_category, y_type, test_size=0.2, random_state=42)

input_layer = keras.Input(shape=(224, 224, 3))

x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)

category_output = layers.Dense(len(category_mapping), activation='softmax', name="category_output")(x)
type_output = layers.Dense(len(type_mapping), activation='softmax', name="type_output")(x)

model = keras.Model(inputs=input_layer, outputs=[category_output, type_output])

model.compile(optimizer='adam',
              loss={'category_output': 'sparse_categorical_crossentropy', 
                    'type_output': 'sparse_categorical_crossentropy'},
              metrics={'category_output': ['accuracy'], 
                       'type_output': ['accuracy']})  


history = model.fit(X_train, 
                    {"category_output": y_train_category, "type_output": y_train_type}, 
                    epochs=7, 
                    validation_data=(X_val, {"category_output": y_val_category, "type_output": y_val_type}),
                    batch_size=32)

model.save("outfit_recommendation_model.h5")

print("✅ Model training completed and saved successfully!")


Epoch 1/7
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 662ms/step - category_output_accuracy: 0.5147 - category_output_loss: 1.7753 - loss: 3.2677 - type_output_accuracy: 0.5308 - type_output_loss: 1.4924 - val_category_output_accuracy: 0.9375 - val_category_output_loss: 0.5816 - val_loss: 1.2484 - val_type_output_accuracy: 0.5833 - val_type_output_loss: 0.6662
Epoch 2/7
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 584ms/step - category_output_accuracy: 0.7268 - category_output_loss: 0.5596 - loss: 1.2132 - type_output_accuracy: 0.6196 - type_output_loss: 0.6536 - val_category_output_accuracy: 0.8750 - val_category_output_loss: 0.3317 - val_loss: 1.3808 - val_type_output_accuracy: 0.4792 - val_type_output_loss: 1.0379
Epoch 3/7
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 578ms/step - category_output_accuracy: 0.8978 - category_output_loss: 0.3074 - loss: 1.1200 - type_output_accuracy: 0.5769 - type_output_loss: 0.8125 - val_category_output_accuracy: 0.9583 - val_category_output_loss: 0.1766 - val_loss: 0.7093 - val_type_output_accurac

✅ Model training completed and saved successfully!


In [ ]:
model = tf.keras.models.load_model("outfit_recommendation_model.h5")

feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-3].output)
feature_extractor_model.build(input_shape=(None, 224, 224, 3))

print("✅ Feature extractor model is ready!")

dataset_folder = "../pre-processing/processed_images"
image_paths = []
dataset_features = []
dataset_categories = []
dataset_types = []

for _, row in df.iterrows():
    img_path = os.path.join(dataset_folder, row["image_filename"])
    
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0  
        img = np.expand_dims(img, axis=0)

        features = feature_extractor_model.predict(img)[0]
        dataset_features.append(features)
        image_paths.append(img_path)
        dataset_categories.append(row["Category_Label"])
        dataset_types.append(row["Type_Label"])

dataset_features = np.array(dataset_features)
np.save("dataset_features.npy", dataset_features)
np.save("image_paths.npy", np.array(image_paths))
np.save("dataset_categories.npy", np.array(dataset_categories))
np.save("dataset_types.npy", np.array(dataset_types))

print("✅ Dataset features & metadata saved successfully!")


✅ Feature extractor model is ready!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━

In [ ]:
dataset_features = np.load("dataset_features.npy")
image_paths = np.load("image_paths.npy")
dataset_categories = np.load("dataset_categories.npy")
dataset_types = np.load("dataset_types.npy")

titles = np.load("titles.npy", allow_pickle=True)
prices = np.load("prices.npy", allow_pickle=True)
links = np.load("links.npy", allow_pickle=True)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0  
    img = np.expand_dims(img, axis=0) 
    return img

def recommend_outfit(query_image_path, top_n=8):
    query_img = preprocess_image(query_image_path)
    query_features = feature_extractor_model.predict(query_img)[0]
    query_category, query_type = model.predict(query_img)
    query_category = np.argmax(query_category)
    query_type = np.argmax(query_type)

    similarities = cosine_similarity([query_features], dataset_features)[0]

    valid_indices = [i for i in range(len(dataset_features)) if dataset_categories[i] == query_category and dataset_types[i] == query_type]

    filtered_similarities = [(i, similarities[i]) for i in valid_indices]
    filtered_similarities = sorted(filtered_similarities, key=lambda x: x[1], reverse=True)[:top_n]

    recommendations = [{
        "image": image_paths[i],
        "title": titles[i],
        "price": prices[i],
        "link": links[i],
        "similarity": round(score, 2)
    } for i, score in filtered_similarities]

    return recommendations

query_img_path = "x.jpeg"
recommendations = recommend_outfit(query_img_path)

print("🎯 Recommended outfits:")
for i, rec in enumerate(recommendations):
    print(f"{i+1}. {rec['title']} - ₹{rec['price']}")
    print(f"   🖼️ Image: {rec['image']}")
    print(f"   🔗 Link: {rec['link']}")
    print(f"   🔍 Similarity Score: {rec['similarity']}")
    print("--------------------------------------------------")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
🎯 Recommended outfits:
1. Verdusa Men's Drop Shoulder Round Neck Half Sleeve T Shirt Basic Solid Loose Tee Tops - ₹25
   🖼️ Image: ../pre-processing/processed_images/product_1741031006_26.jpg
   🔗 Link: https://www.amazon.com/Verdusa-Shoulder-Round-Sleeve-Medium/dp/B0CP3RCTLB/ref=sr_1_25?dib=eyJ2IjoiMSJ9.WZ3oaf_kkJVNJ0Y78Rb9yurfEuQ87d5XMJFYEPeHFq_Mc2cZSsxOXEqvUn4xl3oNGzI7FFYaNs8mwt1p6P5WUJK8j_mcuw4BCoAsAFBxETBOqTS_8ENELvO64HV8AkDOdkYO4_T3-hUQvWaI-ixqFUR1HRNabZMefhlb---F5KaLWaUVgkBpQr9c7dq-ID7AfrFCkLWL_lDYhNNrFpTut8LXuNm8nuqf3VbHu2ESVSUmG04jQL-5QnfZkxfc75YZs2sx1S-UcYVedhynzrVQ6PJBe1NrHTxFrGTmYohnrMU.q0lICjaMYrcvGel694GjCIF0aqgF20XFJi_WLQTwPT4&dib_tag=se&keywords=half+sleeves+shirts+for+men&qid=1741030869&sr=8-25
   🔍 Similarity Score: 0.9900000095367432
--------------------------------------------------
2. Alimens & Gentle Mens Short Sleeve Dress Shirt Solid Casual Button Down Shirts with Pocket for Summer Busin

In [ ]:
print(len(model.layers))

12
